In [2]:
import numpy as np
import cvxopt
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import matplotlib.cm as cmx
import matplotlib.colors as colors

In [3]:
del_t = 0.1
v_init = 0.1
w_init = 0

class agent:
    def __init__(self,rad,start,goal,theta,vmax,wmax):
        self.rad = rad
        self.time = 0
        self.goal = goal
        self.theta = theta
        self.v = v_init
        self.w = w_init
        self.pos = start
        self.vmax = vmax
        self.wmax = wmax
        self.optfl = 0
    def __str__(self):
        return (str(self.time)+" "+str(self.theta)+" "+str(self.v)+" "+str(self.w)+" "+str(self.pos))
    def opt_traj(self):
        self.visualize_traj()
        cost_hist = []
        cost_hist.append(0)
        it = 0
        while (np.linalg.norm(self.pos-self.goal)>0.5):
#             self.visualize_traj()
            if (self.optfl==0):
                theta_new = self.theta + self.w*del_t
                xgi = self.pos[0] + self.v*del_t*np.cos(theta_new) - self.goal[0]
                ygi = self.pos[1] + self.v*del_t*np.sin(theta_new) - self.goal[1]
                p_mat = 2*np.array([[del_t**2,0],
                                  [0,(self.v**2)*(del_t**4)]])
                P = cvxopt.matrix(p_mat,tc='d')
                q_mat = np.array([2*del_t*(xgi*np.cos(theta_new)+ygi*np.sin(theta_new)),
                                 -2*del_t**2*self.v*(xgi*np.sin(theta_new)-ygi*np.cos(theta_new))])
    #             print (p_mat,q_mat)
                Q = cvxopt.matrix(q_mat,tc='d')
                g_mat = np.array([[1,0],[0,1],[0,-1]])
                h = cvxopt.matrix(np.array([self.vmax-self.v,self.wmax-self.w,self.wmax-self.w]),tc='d')
                g = cvxopt.matrix(g_mat,tc='d')
                sol = cvxopt.solvers.qp(P,Q,g,h,options={'show_progress': False})
                v_new = sol['x'][0]
                w_new = sol['x'][1]
                print ("v_new: ",v_new,"w_new: ",w_new)
                it+=1
                x_new = self.pos[0] + (self.v+v_new)*del_t*np.cos(self.theta+(self.w+w_new)*del_t)
                y_new = self.pos[1] + (self.v+v_new)*del_t*np.sin(self.theta+(self.w+w_new)*del_t)
                cost = np.linalg.norm(np.array([x_new,y_new])- self.goal)**2
                prev_cost = cost_hist[-1]
                print (it,cost)
                if (np.linalg.norm(cost-prev_cost)>0.1):
                    self.v += v_new 
                    self.w += w_new
                    cost_hist.append(cost)
                else:
                    self.optfl = 1
                    self.v += v_new 
                    self.w += w_new
            else:
                x_new = self.pos[0] + self.v*del_t*np.cos(self.theta+self.w*del_t)
                y_new = self.pos[1] + self.v*del_t*np.sin(self.theta+self.w*del_t)
                self.pos = [x_new,y_new]
                print (self)
                self.theta = self.theta+w_new*del_t
                self.time+=1
                self.visualize_traj()
                
             
    def visualize_traj(self):
        figure = plt.figure()
        ax = figure.add_subplot(1,1,1)
        robot = matplotlib.patches.Circle(
            (self.pos[0],self.pos[1]),
            radius = self.rad,
            edgecolor='black',
            linewidth=1.0,
            ls='solid',
            alpha=1,
            zorder=2)
        ax.add_patch(robot)
        name = 'data/snap%s.png'%str(self.time)
        ax.plot([self.goal[0]], [self.goal[1]], '*', color="red", markersize =15,linewidth=3.0)
        ax.set_aspect('equal')
        ax.set_xlim(-10.0, 60.0)
        ax.set_ylim(-10.0, 60.0)
        ax.set_xlabel(r'$x (m)$')
        ax.set_ylabel(r'$y (m)$')
        ax.grid(True)
        plt.savefig(name, dpi = 200)
        plt.cla()
        plt.close(figure)
        return figure
    


In [4]:
bot = agent(2,np.array([0,0]),np.array([50,50]),np.pi/4,10,0.5)
bot.opt_traj()

v_new:  9.900000033697594 w_new:  4.441905805252539e-14
1 4859.5786432928735
v_new:  -3.3360618122593634e-08 w_new:  -3.12619132648682e-12
2 4859.578643757994
0 0.7853981633974483 10.000000000336977 -3.0817722684342945e-12 [0.7071067812105933, 0.7071067812101574]
1 0.7853981633971356 10.000000000336977 -3.0817722684342945e-12 [1.4142135624214078, 1.4142135624200938]
2 0.785398163396823 10.000000000336977 -3.0817722684342945e-12 [2.1213203436324433, 2.121320343629809]
3 0.7853981633965104 10.000000000336977 -3.0817722684342945e-12 [2.8284271248437, 2.828427124839303]
4 0.7853981633961977 10.000000000336977 -3.0817722684342945e-12 [3.5355339060551776, 3.535533906048576]
5 0.7853981633958851 10.000000000336977 -3.0817722684342945e-12 [4.242640687266876, 4.242640687257628]
6 0.7853981633955724 10.000000000336977 -3.0817722684342945e-12 [4.949747468478796, 4.949747468466459]
7 0.7853981633952598 10.000000000336977 -3.0817722684342945e-12 [5.656854249690936, 5.656854249675069]
8 0.7853981633